데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)

x_train:

[https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv)

y_train:

[https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv)

x_test:

[https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv)

x_label(평가용) :

[https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv)

데이터 출처 :

[https://www.kaggle.com/datasets/prachi13/customer-analytics](https://www.kaggle.com/datasets/prachi13/customer-analytics)

(참고, 데이터 수정)